In [ ]:
# autoencoder_dynamic.ipynb
# Copyright 2020 André Carrington, Ottawa Hospital Research Institute
# Use is subject to the Apache 2.0 License
# Written by André Carrington

%matplotlib inline
import pandas as pd
import numpy as np
import time

# suppress warnings because they are verbose and distracting
import warnings
warnings.filterwarnings('ignore')

cycle_names    = [
  'cchs-82M0013-E-2001-c1-1-general-file_F1.csv',
  'cchs-82M0013-E-2003-c2-1-GeneralFile_F1.csv',
  'cchs-82M0013-E-2005-c3-1-main-file_F1.csv',
  'CCHS-82M0013-E-2009-2010-Annualcomponent_F1.csv',
  'CCHS-82M0013-E-2010-AnnualComponent_F1.csv',
  'cchs-82M0013-E-2011-2012-Annual-component_F1.csv',
  'cchs-82M0013-E-2012-Annual-component_F1.csv',
  'cchs-82M0013-E-2013-2014-Annual-component_F1.csv',
  'cchs-82M0013-E-2014-Annual-component_F1.csv',
  'cchs-E-2007-2008-AnnualComponent_F1.csv'
];

test_run_text     = input('experiment number: ')
test_cycles_text  = input('test_cycles (e.g., 1): ')
encoding_dim_text = input('encoded_dimensionality (e.g., 50): ')
encoded_mult_text = input('encoded_multiples (e.g., 0,8,4,1; where 0 is dropout): ') # [8 4 1]
main_activ        = input('main activation function (e.g., relu, sigmoid, tanh): ')
output_activ      = input('output activation function (e.g., sigmoid, tanh, relu): ')
dropout_rate_text = input('dropout rate if applicable (e.g., 0.2): ')
train_noise_text  = input('training noise (size of middle 4 standard deviations, e.g., 0 or 0.3): ')
val_noise_text    = input('validation noise (size of middle 4 standard deviations, e.g., 0 or 0.3): ')
optimizer         = input('optimizer (e.g., Nadam, Adam, Adamax, Adadelta, Adagrad, RMSprop, SGD): ')
objective         = input('objective to minimize (e.g., MSE, MAE): ')
max_epochs_text   = input('max_epochs (e.g., 400): ')
patience_text     = input('patience (e.g., 15): ')

tic               = time.perf_counter()

encoding_dim      = int(encoding_dim_text)
test_cycles       = [int(i) for i in list(test_cycles_text.split(","))] 
encoded_multiples = [int(i) for i in list(encoded_mult_text.split(","))] 
dropout_rate      = float(dropout_rate_text)
train_noise       = float(train_noise_text)
val_noise         = float(val_noise_text)
# optimizer
# objective
max_epochs        = int(max_epochs_text)
patience_val      = int(patience_text)

# based on the metric we choose to optimize
# get a list of the other metrics (into mets) to also report on
mets = []
for met in ['MSE','MAE','binary_crossentropy','cosine_similarity']:
    if met == objective:
        continue
    mets = mets + [met]
#endfor

#test_cycles      = list(test_cycles_text.split(","))
#encoded_multiples= range(2,19)/2 # (2 to 18)/2 = 1,1.5,2,...,8.5,9

# Import data into a Pandas Dataframe (df)
temp_df        = {};
in_path        = 'C:\\Users\\amcarrin\\Desktop\\Andre2\\PUMF_data\\'
#in_path= 'PUMF_data/';
#in_path= '//Users/andrecarrington/Documents/0 OHRI/Projects/Montfort CDSS/PUMF/';
for i in range(0,len(test_cycles)):
    #in_fn     = in_path + cycle_names[int(test_cycles[i])-1]
    in_fn      = in_path + cycle_names[test_cycles[i]-1]
    print       (in_fn)
    temp_df[i] = pd.read_csv(in_fn)
    print(temp_df[i].shape)
    
toc            = time.perf_counter()
print(f"Loaded files in {toc - tic:0.1f} seconds")

# only use the first cycle for now
in_df   = temp_df[0]

# drop the administrative record (sequence) number
in_df = in_df.drop(columns=['ADMA_RNO'])

# # load a dataframe with a column listing the relevant variables 
# temp_df  = pd.read_csv("relevant_variables.csv",header=None)
# rel_vars = temp_df.iloc[:,0] # convert df to series
# temp_df  = in_df.copy()
# in_df    = temp_df[rel_vars] # filter in_df to contain only relevant variables 


In [ ]:
tic = time.perf_counter()

from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

toc = time.perf_counter()
print(f"Imported tensorflow and keras in {toc - tic:0.1f} seconds")

In [ ]:
def leakyrelu(x, alpha=0.01): 
    if (x>0):
        return np.maximum(0, x)
    else:
        return x*alpha
    #endif
#enddef

In [ ]:
tic               = time.perf_counter()

# _dim for dimensions
in_feature_dim    = in_df.shape[1] # 613 
in_instance_dim   = in_df.shape[0] # 130880

# Example of a 1+1 layer autoencoder, where xi,yi are defined in code further below
# x = Dense(encoding_dim,   activation='relu'   )(xi) # encoder
# a = Dense(in_feature_dim, activation='sigmoid')(x)  # autoencoder
# y = Dense(in_feature_dim, activation='sigmoid')(yi) # decoder

# Example of a 2+2 layer autoencoder, where xi,yi are defined in code further below
# x = Dense(2*encoding_dim, activation='relu'   )(xi)
# x = Dense(  encoding_dim, activation='relu'   )(x)  # encoder
# a = Dense(2*encoding_dim, activation='relu'   )(x)
# a = Dense(in_feature_dim, activation='sigmoid')(a)  # autoencoder
# y = Dense(2*encoding_dim, activation='relu'   )(yi)
# y = Dense(in_feature_dim, activation='sigmoid')(y)  # decoder

# check encoded_multiples, e.g., [8,4,1]
if len(encoded_multiples)==0:
    raise ValueError('At least one multiple is required')
if encoded_multiples[-1] != 1:
    raise ValueError('The last multiple should be 1')
    
# Build the encoder x
xi          = Input(shape=(in_feature_dim,)) # input to encoder x and autoencoder a
x           = {}
last        = len(encoded_multiples)-1
for m in range(last+1):
    mult = encoded_multiples[m]
    if m == 0: # first_layer
        layer_input = xi
    else:
        layer_input = x
    #endif
    if   mult == 0:   # indicates a dropout layer
        x   = Dropout(dropout_rate)(layer_input)
    elif mult >  0:   # multiples greater than 0 indicate a dense layer
        x   = Dense(mult*encoding_dim, activation=main_activ)(layer_input)
    #endif
#endfor
    
encoder_m   = Model(xi, x)
    
# Build the decoder y and autoencoder a
decoded_multiples = encoded_multiples.copy()   # [0,8,4,1]
if decoded_multiples[0] == 0:                  # for decoder, do not "relect" the encoder's first dropout layer
    decoded_multiples = decoded_multiples[1:]  # [8,4,1] 
decoded_multiples.reverse()                    # [1,4,8] reflect/reverse. note: a last layer will be automatically added 
decoded_multiples = decoded_multiples[1:]      # [4,8]   note: this array can be zero length
yi          = Input(shape=(encoding_dim,))     # input to decoder y
y           = {}
a           = {}
last        = len(decoded_multiples)-1
#last        = len(decoded_multiples)
if len(decoded_multiples)==0: # single layer decoder
    y       = Dense(in_feature_dim, activation=output_activ)(yi)
    a       = Dense(in_feature_dim, activation=output_activ)(x)    
else:
    for m in range(last+1):
        mult = decoded_multiples[m]
        if m == 0: # first_layer
            layer_input  = yi
            layer_inputa = x
        else:
            layer_input  = y
            layer_inputa = a
        #endif            
        if   mult == 0:   # indicates a dropout layer
            y   = Dropout(dropout_rate)(layer_input)
            a   = Dropout(dropout_rate)(layer_inputa)
        elif mult >  0:   # multiples greater than 0 indicate a dense layer
            # add layer
            y = Dense(mult*encoding_dim, activation=main_activ)(layer_input)
            a = Dense(mult*encoding_dim, activation=main_activ)(layer_inputa)
            if m  == last: # after last defined layer, add a final output layer too, automatically
                y = Dense(in_feature_dim, activation=output_activ)(y)
                a = Dense(in_feature_dim, activation=output_activ)(a)
            #endif
        #endif
    #endfor
#endif

decoder_m     = Model(yi, y)
autoencoder_m = Model(xi, a)

autoencoder_m.compile(optimizer=optimizer, loss=objective, metrics=mets)
# autoencoder_m.compile(optimizer='Nadam', 
#                       loss='MSE',
#                       metrics=['cosine_similarity','MAE','binary_crossentropy'])
# autoencoder_m.compile(optimizer='adadelta', 
#                      loss='MSE',
#                      metrics=['cosine_similarity','MAE','binary_crossentropy'])

toc           = time.perf_counter()
print(f"Compiled models in {toc - tic:0.1f} seconds")


In [ ]:
print(autoencoder_m.summary())

In [ ]:
from sklearn.preprocessing   import MinMaxScaler
from sklearn.model_selection import KFold
import tensorflow as tf

tic                 = time.perf_counter()

# "normalize" to positive range using min-max standardization to [0,1]
# result is a numpy.ndarray (nd), convert it back to a Pandas dataframe (df)
temp_df             = in_df.copy()
scaler              = MinMaxScaler()  # obtain the scaler for later denormalization
in_std_nd           = scaler.fit_transform(temp_df)
in_std_df           = pd.DataFrame(in_std_nd)

toc                 = time.perf_counter()
print(f"Normalized data in {toc - tic:0.1f} seconds")


In [ ]:
tic           = time.perf_counter()

if train_noise == 0:
    in_noi_df   = in_std_df
else:
    in_noi_df   = in_std_df + \
                train_noise * np.random.normal(loc=0.0, scale=1/4, size=in_std_df.shape)
    fix_max_index = in_noi_df > 1
    fix_min_index = in_noi_df < 0
    in_noi_df[fix_max_index]  = 1
    in_noi_df[fix_min_index]  = 0

if   val_noise == train_noise:
    val_noi_df = in_noi_df.copy()
elif val_noise == 0:
    val_noi_df = in_std_df.copy()
else:
    val_noi_df = in_std_df + \
                 val_noise * np.random.normal(loc=0.0, scale=1/4, size=in_std_df.shape)
    fix_max_index = val_noi_df > 1
    fix_min_index = val_noi_df < 0
    val_noi_df[fix_max_index]  = 1
    val_noi_df[fix_min_index]  = 0
    
toc           = time.perf_counter()
print(f"Created noisy (standardized) data in {toc - tic:0.1f} seconds")

In [ ]:
tic                 = time.perf_counter()

# split data into 70% training, 20% validation 10% hold out testing
# use KFold to get 10 mutually exclusive folds with shuffling
# note: not being used for cross-validation (at this time) -- just for train / validation / test split
kf=KFold(n_splits=10, random_state=26, shuffle=True)

i = 0
train_index = list()
val_index   = list()
test_index  = list()
for unused, val_temp in kf.split(in_df):
    print(val_temp)
    i = i + 1
    if i < 9:
        train_index.extend(val_temp.tolist())
    if i==8 or i==9:
        val_index.extend(val_temp.tolist())
    if i==10:
        test_index = val_temp.tolist()

print(len(train_index))

train_in_df     =  in_df.iloc[train_index,:]
val_in_df       =  in_df.iloc[val_index  ,:]
test_in_df      =  in_df.iloc[test_index ,:]
    
train_in_std_df =  in_std_df.iloc[train_index,:]
val_in_std_df   =  in_std_df.iloc[val_index  ,:]
test_in_std_df  =  in_std_df.iloc[test_index ,:]
    
train_in_noi_df =  in_noi_df.iloc[train_index,:]
val_in_noi_df   = val_noi_df.iloc[val_index  ,:]
test_in_noi_df  =  in_noi_df.iloc[test_index ,:]
    
toc                 = time.perf_counter()
print(f"Shuffled and split data in {toc - tic:0.1f} seconds")


In [ ]:
tic                 = time.perf_counter()

out_fn              = in_path + 'val_in_df_' + test_run_text + '.csv'
val_in_df.to_csv(out_fn, index = False, header=True)

toc                 = time.perf_counter()
print(f"Wrote non-standardized data to file in {toc - tic:0.1f} seconds")

# show the non standardized validation set 
# to compare against the decoder result that comes later
val_in_df

In [ ]:
# show the standardized validation set 
# to compare against the decoder result that comes later
val_in_std_df

In [ ]:
# learn the autoencoder on the standardized training data, validated with standardized validation data

tic            = time.perf_counter()

# prevent overfitting with EarlyStopping (es) at minimum validation loss, 
# allowing for patience=15 (for example) epochs of no better value 
from tensorflow.keras.callbacks import EarlyStopping
es             = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience_val)

# run the autoencoder with a callback to es, and capturing history _h
loss_metric_h  = autoencoder_m.fit(train_in_noi_df, train_in_std_df, epochs=max_epochs, batch_size=256,
                                   shuffle=True, validation_data=(val_in_noi_df, val_in_std_df),
                                   callbacks=[es])

toc            = time.perf_counter()
print(f"Trained and validated autoencoder in {toc - tic:0.1f} seconds")

tic            = time.perf_counter()

# convert the history dict to a pandas DataFrame:     
loss_metric_df = pd.DataFrame(loss_metric_h.history) 

# save to csv: 
hist_csv_file  = in_path + 'loss_metric_history_' + test_run_text + '.csv'
with open(hist_csv_file, mode='w') as f:
    loss_metric_df.to_csv(f)

toc            = time.perf_counter()
print(f"Wrote loss_metric data to file in {toc - tic:0.1f} seconds")

In [ ]:
tic             = time.perf_counter()

# encode and decode data using the validation set
encoded_imgs_nd = encoder_m.predict(val_in_std_df)
decoded_imgs_nd = decoder_m.predict(encoded_imgs_nd)
print('encoded_imgs_nd: ',type(encoded_imgs_nd),encoded_imgs_nd.shape)
print('decoded_imgs_nd: ',type(decoded_imgs_nd),decoded_imgs_nd.shape)

toc             = time.perf_counter()
print(f"Encoded and decoded data in {toc - tic:0.1f} seconds")

# show the standardized output, val_out_std_df
val_out_std_df  = pd.DataFrame(decoded_imgs_nd)
val_out_std_df

In [ ]:
tic             = time.perf_counter()

# get and show the output (removing the standardization)
val_out_nd      = scaler.inverse_transform(decoded_imgs_nd)
val_out_df      = pd.DataFrame(val_out_nd)

out_fn          = in_path + 'val_out_df_' + test_run_text + '.csv'
val_out_df.to_csv(out_fn, index = False, header=True)

toc             = time.perf_counter()
print(f"Denormalized and saved data in {toc - tic:0.1f} seconds")

val_out_df